In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from scipy import stats
file_path = 'train_data_merged.csv'

# 读取CSV文件
df = pd.read_csv(file_path)

# 显示前几行数据
print(df.head())


             LOG_ID            PAT_ID               MRN  Cardiovascular   LOS  \
0  d754f06a7d973a26  837532f458d9c660  0b8de903ea63082a             0.0   1.0   
1  5a95970d5c8fc355  fd18a41f136ed278  0a8b72c1cec4ae47             0.0  10.0   
2  138d44a06cf4b57f  fd18a41f136ed278  0a8b72c1cec4ae47             0.0  10.0   
3  82372dc4703ea1b0  648781c95d863ec6  e0cb9244fedd1ac7             0.0   3.0   
4  4b6aabaf62e85c0a  d1d7b9ddc4962cdf  3918b79e03cefe27             0.0   3.0   

   ICU_ADMIN_FLAG      HEIGHT    WEIGHT  SEX  ASA_RATING_C  ...  \
0               0  187.960000   92.6991    1           2.0  ...   
1               1  168.251459  100.9530    1           3.0  ...   
2               1  168.251459  100.9530    1           3.0  ...   
3               1  160.020000   70.1064    0           2.0  ...   
4               1  168.251459   86.3922    1           2.0  ...   

   diastolic_peak  pulse_amplitude  arc_peak_frequency  arc_spectral_energy  \
0      432.481028      3252.798

In [3]:
df2 = pd.read_csv('features_part2.csv')
print(df2)

                LOG_ID               MRN  Cardiovascular Calcium  \
0     8c615f6805325d06  692072abc16db15d             1.0       0   
1     0ecf38ed306338f6  4c29aee08612d793             1.0       0   
2     595439a55deda0d5  71d77695a69b26d2             1.0       L   
3     36662c223daeac3a  08b0d6a24cf382ad             1.0       L   
4     1383c875d3b6ffbd  4c29aee08612d793             1.0       0   
...                ...               ...             ...     ...   
1599  d397b30aa594c59a  69ad863d072730c3             0.0       N   
1600  73b71327357c3dc2  75f287c191a0fc0d             0.0       L   
1601  586b681a1822a032  0edc43c1a746b88a             0.0       N   
1602  3c146c502e1e249d  2408210232a3e06b             0.0       0   
1603  3333ade5695c0b55  c280ccc5e979b4ef             0.0       L   

     Erythrocyte distribution width Erythrocyte mean corpuscular hemoglobin  \
0                                 0                                       0   
1                        

In [10]:
# 假设 df2 已经存在并且包含所需的列

# 创建一个映射，将 'N', 'L', 'H' 替换为 1, 2, 3
replace_map = {'N': 1, 'L': 2, 'H': 3}

# 列表中需要替换的列
columns_to_replace = [
    'Calcium', 'Erythrocyte distribution width','Erythrocyte mean corpuscular hemoglobin',
    'Erythrocyte mean corpuscular hemoglobin concentration',
    'Erythrocyte mean corpuscular volume', 'Erythrocytes', 
    'Hematocrit', 'Hemoglobin'
]

# 对指定的列进行替换
df2[columns_to_replace] = df2[columns_to_replace].replace(replace_map)

# 查看替换后的数据
print(df2)


                LOG_ID               MRN  Cardiovascular Calcium  \
0     8c615f6805325d06  692072abc16db15d             1.0       0   
1     0ecf38ed306338f6  4c29aee08612d793             1.0       0   
2     595439a55deda0d5  71d77695a69b26d2             1.0       2   
3     36662c223daeac3a  08b0d6a24cf382ad             1.0       2   
4     1383c875d3b6ffbd  4c29aee08612d793             1.0       0   
...                ...               ...             ...     ...   
1599  d397b30aa594c59a  69ad863d072730c3             0.0       1   
1600  73b71327357c3dc2  75f287c191a0fc0d             0.0       2   
1601  586b681a1822a032  0edc43c1a746b88a             0.0       1   
1602  3c146c502e1e249d  2408210232a3e06b             0.0       0   
1603  3333ade5695c0b55  c280ccc5e979b4ef             0.0       2   

     Erythrocyte distribution width Erythrocyte mean corpuscular hemoglobin  \
0                                 0                                       0   
1                        

In [12]:
import pandas as pd

# 假设 df1 和 df2 已经存在
# 例如，你可以从 CSV 文件中读取这两个数据框：
# df1 = pd.read_csv('path_to_df1.csv')
# df2 = pd.read_csv('path_to_df2.csv')

# 使用 pandas 的 merge 函数根据 LOG_ID 进行合并
df_merged = pd.merge(df, df2, on=['LOG_ID', 'MRN','Cardiovascular'], how='left')

# 查看合并后的数据
print(df_merged)
df_final=df_merged.drop(columns=['LOG_ID','PAT_ID','MRN'])

                LOG_ID            PAT_ID               MRN  Cardiovascular  \
0     d754f06a7d973a26  837532f458d9c660  0b8de903ea63082a             0.0   
1     5a95970d5c8fc355  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
2     138d44a06cf4b57f  fd18a41f136ed278  0a8b72c1cec4ae47             0.0   
3     82372dc4703ea1b0  648781c95d863ec6  e0cb9244fedd1ac7             0.0   
4     4b6aabaf62e85c0a  d1d7b9ddc4962cdf  3918b79e03cefe27             0.0   
...                ...               ...               ...             ...   
1599  8116aa4a100ceee8  478a3cd8857cc055  72accda5075f7a39             1.0   
1600  3c146c502e1e249d  fa6ec9ad3f5f1466  2408210232a3e06b             0.0   
1601  cecd35a626c55d0c  b9878cd048a80cb3  5b9cf2e5349171e8             1.0   
1602  3333ade5695c0b55  2f091eb1b00105d4  c280ccc5e979b4ef             0.0   
1603  c34b85c0cf941d8a  b5838d1cdf37c09c  d7946ede59551209             1.0   

       LOS  ICU_ADMIN_FLAG      HEIGHT    WEIGHT  SEX  ASA_RATI

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

# 假设 df1 是你的数据集
# 将数据集分为特征X和目标y
X = df_final.drop('Cardiovascular', axis=1)  # 'target_column' 是你要预测的标签列
y = df_final['Cardiovascular']

# 将数据集划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 初始化标准化工具 (StandardScaler) 进行归一化
scaler = StandardScaler()

# 对训练集和测试集进行归一化
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 初始化SVM分类器
svm_model = SVC()

# 使用归一化后的数据训练模型
svm_model.fit(X_train_scaled, y_train)

# 使用模型进行预测
y_pred = svm_model.predict(X_test_scaled)

# 计算并显示准确率、精确率和召回率
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')  # 'binary' 适用于二分类问题
recall = recall_score(y_test, y_pred, average='binary')

# 显示结果
print(f"准确率 (Accuracy): {accuracy}")
print(f"精确率 (Precision): {precision}")
print(f"召回率 (Recall): {recall}")

准确率 (Accuracy): 0.8879668049792531
精确率 (Precision): 0.922077922077922
召回率 (Recall): 0.5966386554621849


In [16]:
df_merged.to_csv('more_features.csv',index=False)